# BCB546X Python Assignment Jupyter Notebook
## Christian Montes-Serey

Loading (importing) the required packages

In [1]:
from Bio import SeqIO
from Bio.Data import CodonTable
import pandas as pd

## First function ##
Dr. X: this gets sequences.
Please finish documenting this function with comments, or in notebook markdown text.
Your descriptions of all functions should contain information about what the function does, as well as information about the return types and arguments.

In [2]:
def get_sequences_from_file(fasta_fn):
    sequence_data_dict = {}
    for record in SeqIO.parse(fasta_fn, "fasta"):
        description = record.description.split()
        species_name = description[1] + " " + description[2]
        sequence_data_dict[species_name] = record.seq
    return(sequence_data_dict)

**First function explained:** We are defining (creating) a function called `get_seqences_from_file`. We have to provide the `fasta_fn` argument when using this function. This is, providing a **fasta** format file.  
The `get_seqences_from_file` function does the following:  
1. Creates an empty dictionary object, called `sequence_data_dict`  
2. Starts a `for` loop that goes through each record in the provided **fasta** file (the `SeqIO.parse` function is used to access the file content and the `record` variable is used to store the parsed record). Each loop do:  
2.1. 

In [12]:
for asdf in SeqIO.parse("penguins_cytb.fasta", "fasta"):
    print(asdf.description.split()[1] + " " + asdf.description.split()[2])
    print(asdf.seq)

Aptenodytes forsteri
ATGGCCCCAAATCTCCGAAAATCCCATCCCCTCCTAAAAATAATTAATAACTCCCTAATCGACCTGCCCACCCCATCAAACATCTCTGCCTGATGAAACTTCGGATCTCTCCTAGGCATCTGCCTAACTACACAAATTTTAACCGGCCTCCTACTAGCTATACACTACACTGCAGACACAACCCTAGCCTTCTCCTCAGTCGCCCACACATGCCGAAACGTACAGTACGGCTGACTGATCCGCAACCTACATGCAAACGGAGCATCATTCTTCTTCATCTGCATCTATCTCCACATTGGCCGTGGATTTTACTATGGCTCCTATCTATACAAAGAAACCTGAAACACAGGCATTATCCTCCTACTCACCCTCATGGCAACCGCCTTCGTAGGCTACGTCCTACCATGAGGACAAATATCTTTCTGAGGAGCCACAGTCATTACCAACTTATTCTCAGCCATCCCTTACATTGGCCAAACCCTCGTAGAATGGACCTGAGGTGGCTTTTCAGTAGACAACCCCACATTAACCCGATTTTTCGCACTACACTTCCTCCTTCCCTTCATAATCGCAGGCCTCACCCTCATCCACCTCACCTTCCTCCACGAATCAGGCTCAAATAACCCACTGGGCATCGTAGCTAACTCCGATAAAATCCCATTCCACCCCTACTACTCCACAAAAGACATCCTAGGATTCGCACTCATACTTCTCCCACTAACAACCCTTGCCCTATTCTCCCCCAACCTACTAGGAGACCCAGAAAACTTCACCCCAGCAAACCCACTAGTCACACCCCCACACATCAAACCAGAATGATACTTCCTATTTGCATACGCTATCCTACGCTCAATCCCCAACAAACTAGGAGGAGTCCTTGCCCTAGCAGCATCCGTGCTAATCCTATTCCTAATCCCTCTCCTCCACAAATCCAAGCAACGCACAATAGCTTTCCGCCCTCTCTCCCAACTCCTATTCT

In [4]:
asdf

<generator object parse at 0x0000022F7AC71AF0>

In [5]:
test = asdf.description.split()

AttributeError: 'generator' object has no attribute 'description'